In [4]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

# Load Data
train = pd.read_csv("fashion-mnist_train.csv")
test = pd.read_csv("fashion-mnist_test.csv")

# Data Preprocessing
X_train = train.iloc[:, 1:].values.reshape(-1, 28, 28, 1) / 255.0
X_test = test.iloc[:, 1:].values.reshape(-1, 28, 28, 1) / 255.0
Y_train = to_categorical(train['label'].values)
Y_test = to_categorical(test['label'].values)

# Model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.25),
    Dense(10, activation='softmax')  # 10 here is same as number of labels in dataset
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train-test split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, stratify=Y_train, test_size=0.2, random_state=66)

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True)

# Model training
history = model.fit(X_train, Y_train, batch_size=64, epochs=2, validation_data=(X_val, Y_val),
                    callbacks=[reduce_lr, early_stopping, checkpoint])
# Evaluation


test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=0)
print("Test loss:", test_loss)
print(f"Test accuracy:{test_accuracy*100:.2f}%" )

Test loss: 0.29153043031692505
Test accuracy:89.58%
